## Cassava Leaf Disease Classification

I built an ensemble model using Keras's ResNet152 and EfficientNetB3.  
The result was 0.8777, which was not very high, but I will publish the notebook for information sharing.

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from fastai.vision.all import *

# Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Create some folders for ImageDataGenerator
ImageDataGenerator needs to allocate images to each class folder in advance, so we need to create a folder for each class.

In [ ]:
images_folder = ['CBB', 'CBSD', 'CGM', 'CMD', 'Healthy']

train_folder_path = './train_images'
val_folder_path = './val_images'

#Create some folders
for img_f in images_folder:
    os.makedirs(train_folder_path + '/'+ img_f, exist_ok=True)
    os.makedirs(val_folder_path + '/'+ img_f, exist_ok=True)
    
print("Train folder:", os.listdir(train_folder_path))
print("Validation folder:", os.listdir(val_folder_path))

### Check "train.csv"

In [ ]:
dataset_path = Path('../input/cassava-leaf-disease-classification')

In [ ]:
train_df = pd.read_csv(dataset_path/'train.csv')
train_df.head()

### Check the number of pictures for each class

In [ ]:
n_picture = []
for l in range(5):    
    n_picture.append(len(train_df[train_df['label']==l]))
print("Number of pictures:", n_picture)
print("Total:", sum(n_picture))

In [ ]:
plt.bar(images_folder, n_picture)

### Copy image files to each folder
Divide the image files into the created folders at a ratio of 8:2 for use with ImageDataGenerator.  
If you have some images you don't want to use, add the filename to ‘avoid_list’.  
Example: avoid_list = ['1000015157.jpg', '1000201771.jpg']

In [ ]:
# Put the images you want to exclude in the list
avoid_list = []

# train_test_split
total_size = sum(n_picture)
ratio = 0.8
total_train = int(total_size*ratio)
total_val = int(total_size-total_train)

for l in range(5):
    file_count = 0
    i = 0                    
    file_count = 0
    
    # validation data
    while i < int(n_picture[l]*ratio):
        file_name = train_df[train_df['label']==l].loc[train_df[train_df['label']==l].index[i]]['image_id']
        in_path = dataset_path/'train_images'/file_name
        out_path = val_folder_path + '/' + images_folder[l] + '/' + file_name
        shutil.copyfile(in_path, out_path)
        _, _, files = next(os.walk(val_folder_path + '/' + images_folder[l]))
        file_count = len(files)
        i += 1
        
    # train data
    while i < n_picture[l]:
        file_name = train_df[train_df['label']==l].loc[train_df[train_df['label']==l].index[i]]['image_id']
        #Use avoid_list only for train data
        if not file_name in avoid_list:
            in_path = dataset_path/'train_images'/file_name
            out_path = train_folder_path + '/' + images_folder[l] + '/' + file_name
            shutil.copyfile(in_path, out_path)
            _, _, files = next(os.walk(train_folder_path + '/' + images_folder[l]))
            file_count = len(files)
        i += 1

### Data augmentation


In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255,
                                        rotation_range=360,
                                        width_shift_range=0.1,
                                        height_shift_range=0.1,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        zoom_range=0.5,
                                        shear_range=0.2,
                                        brightness_range=[0.5,1.0],
                                        channel_shift_range=100,
                                        fill_mode = 'nearest')

In [ ]:
validation_image_generator = ImageDataGenerator(rescale=1./255) # 検証データのジェネレータ

In [ ]:
BATCH_SIZE = 8
epochs = 20

IMG_HEIGHT = 512
IMG_WIDTH = 512

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=train_folder_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           classes = images_folder)

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=val_folder_path,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical',
                                                              classes = images_folder)

In [ ]:
print("Train generator's label:\n", train_data_gen.class_indices)
print("Validation generator's label:\n", val_data_gen.class_indices)

### Check the picture

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification?hl=ja
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 7, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(7)]
plotImages(augmented_images)
augmented_images = [train_data_gen[1][0][0] for i in range(7)]
plotImages(augmented_images)
augmented_images = [train_data_gen[2][0][0] for i in range(7)]
plotImages(augmented_images)
augmented_images = [train_data_gen[3][0][0] for i in range(7)]
plotImages(augmented_images)
augmented_images = [train_data_gen[4][0][0] for i in range(7)]
plotImages(augmented_images)

# Training

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.applications import ResNet152, EfficientNetB3
from tensorflow.keras import models, layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adamax
from keras.losses import CategoricalCrossentropy


### Models
reference  
https://www.kaggle.com/bununtadiresmenmor/starter-keras-efficientnet

In [ ]:
def modelResNet152():
    
    model = models.Sequential()
    model.add(ResNet152(include_top = False, 
                      weights='imagenet',
                      #weights = "../input/resnet152traind/resnet152_weights_tf_dim_ordering_tf_kernels_notop.h5",
                      input_shape=(IMG_HEIGHT,IMG_WIDTH, 3)))
    
    model.add(Dropout(0.5))
    model.add(layers.GlobalAveragePooling2D())
    
    #additional
    model.add(layers.Dense(1024, activation = 'relu'))
    model.add(Dropout(0.25))
    
    model.add(layers.Dense(5, activation = "softmax"))
    
    return model 
    

In [ ]:
def modelEfficientNetB3():

    model = models.Sequential()
    model.add(EfficientNetB3(include_top = False, 
                            weights = 'imagenet',
                            #weights = "../input/effib3trained/efficientnetb3_notop.h5",
                            input_shape=(IMG_HEIGHT,IMG_WIDTH, 3)))
    
    model.add(Dropout(0.8))
    model.add(layers.GlobalAveragePooling2D())
    
    #additional
    model.add(layers.Dense(512, activation = 'relu'))
    model.add(Dropout(0.4))
    model.add(layers.Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
 
    model.add(layers.Dense(5, activation = "softmax"))
    
    return model 

In [ ]:
model_res152 = modelResNet152()

In [ ]:
model_effiB3 = modelEfficientNetB3()

In [ ]:
model_res152.summary()

In [ ]:
model_effiB3.summary()

### Callbacks
I used the following three items for the callback.
- ModelCheckpoint  
    If the minimum value of val_loss is updated, it will save the model for each epoch.
- EarlyStopping  
     If val_loss cannot be updated for 7 consecutive epochs, training will end.
- ReduceLROnPlateau  
     If val_loss cannot be updated for 2 consecutive epochs, the learning rate will be multiplied by 0.1.

In [ ]:
model_checkpoint_resnet = ModelCheckpoint(
                            "./checkpoint_resnet.h5",
                            monitor = "val_loss",
                            verbose = 1,
                            save_best_only = True,
                            save_weights_only = False,
                            mode = "min")

In [ ]:
model_checkpoint_effinet = ModelCheckpoint(
                            "./checkpoint_effinet.h5",
                            monitor = "val_loss",
                            verbose = 1,
                            save_best_only = True,
                            save_weights_only = False,
                            mode = "min")

In [ ]:
early_stop = EarlyStopping(
                            monitor = "val_loss",
                            min_delta=0.001,
                            patience=7,
                            verbose=1,
                            mode="min",
                            restore_best_weights=False)

In [ ]:
reduce_lr = ReduceLROnPlateau(
                            monitor="val_loss",
                            factor=0.1,
                            patience=2,
                            verbose=1,
                            mode="min",
                            min_delta=0.0001)

### Compile

In [ ]:
model_res152.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
            loss = CategoricalCrossentropy(label_smoothing=0.3,reduction="auto",name="categorical_crossentropy"),
            metrics = ["accuracy"])

In [ ]:
model_effiB3.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
            loss = CategoricalCrossentropy(label_smoothing=0.3,reduction="auto",name="categorical_crossentropy"),
            metrics = ["accuracy"])

### Training
It takes 6 hours on the GPU.

In [ ]:
epochs = 15

In [ ]:
history_res152 = model_res152.fit_generator(
                    train_data_gen,
                    steps_per_epoch=None,
                    epochs=epochs,
                    validation_data=val_data_gen,
                    validation_steps=None,
                    callbacks = [model_checkpoint_resnet,early_stop,reduce_lr]
)

In [ ]:
epochs = 10

In [ ]:
history_effiB3 = model_effiB3.fit_generator(
                    train_data_gen,
                    steps_per_epoch=None,
                    epochs=epochs,
                    validation_data=val_data_gen,
                    validation_steps=None,
                    callbacks = [model_checkpoint_effinet,early_stop,reduce_lr]
)

In [ ]:
#saving the models
model_res152.save('saved_model_resnet.h5')
model_effiB3.save('saved_model_effinet.h5')

In [ ]:
acc = history_res152.history['accuracy']
val_acc = history_res152.history['val_accuracy']

loss = history_res152.history['loss']
val_loss = history_res152.history['val_loss']

epochs_range = range(len(acc))

plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()

plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
acc = history_effiB3.history['accuracy']
val_acc = history_effiB3.history['val_accuracy']

loss = history_effiB3.history['loss']
val_loss = history_effiB3.history['val_loss']

epochs_range = range(len(acc))

plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()

plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Predict

I submitted the following content as a Predict Notebook separately from the Training Notebook.  
If you want to use the model saved in the training notebook, first upload the saved model from "+ Add Data".  
Then, specify the path of the file in the argument of load_model.

In [ ]:
import numpy as np
import os
import pandas as pd
from fastai.vision.all import *

In [ ]:
import tensorflow as tf
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Load the saved model

In [ ]:
#Choose either
model_res_pred = tf.keras.models.load_model('./saved_model_resnet.h5')
#model_res_pred = tf.keras.models.load_model('./checkpoint_resnet.h5')

#Check the architecture
model_res_pred.summary()

In [ ]:
#Choose either
model_effi_pred = tf.keras.models.load_model('./saved_model_effinet.h5')
#model_effi_pred = tf.keras.models.load_model('./checkpoint_effinet.h5')

#Check the architecture
model_effi_pred.summary()

### Creating a submit file

In [ ]:
dataset_path = Path('../input/cassava-leaf-disease-classification')

Check the sample submission file

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df

Creat a folder for test files

In [ ]:
test_folder_path = './test_images'

if not os.path.exists(test_folder_path):
    os.mkdir(test_folder_path) 

Copy the test file together with the folder 

In [ ]:
test_ds_path = '../input/cassava-leaf-disease-classification/test_images'
test_dir_path = './test_images/all_classes'

if not os.path.exists(test_dir_path):
    shutil.copytree(test_ds_path, test_dir_path)

In [ ]:
_, _, files = next(os.walk(test_dir_path))
file_count = len(files)

print("Number of pictures: ", file_count)
print("Picture name: ", files)

In [ ]:
test_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
test_folder_path = './test_images'

In [ ]:
#IMG_HEIGHT = 512
#IMG_WIDTH = 512

In [ ]:
test_generator = test_image_generator.flow_from_directory(
                                        directory=test_folder_path,
                                        target_size=(IMG_HEIGHT, IMG_WIDTH),
                                        class_mode=None,
                                        shuffle=False
                                        )

In [ ]:
pred_res = model_res_pred.predict_generator(test_generator, verbose=1)
print(pred_res)

In [ ]:
pred_effi = model_effi_pred.predict_generator(test_generator, verbose=1)
print(pred_effi)

### Ensemble
Combine the output results of ResNet and EfficientNet.

In [ ]:
total_pred = pred_res*0.5 + pred_effi*0.5
print("Ensemble predict:", total_pred)

In [ ]:
predicted_class_indices = np.argmax(total_pred, axis=1)
print("Predicted class indices:", predicted_class_indices)

labels_dict = ({'CBB': 0, 'CBSD': 1, 'CGM': 2, 'CMD': 3, 'Healthy': 4})
labels = dict((v,k) for k,v in labels_dict.items())
predictions = [labels[k] for k in predicted_class_indices]
print("Predicted class:", predictions)

### Submission

In [ ]:
# Submission dataframe
submit_ID = sample_df.loc[:]['image_id']
submit_TARGET = pd.DataFrame(predicted_class_indices)

submission_df = pd.concat([submit_ID, submit_TARGET], axis=1)
submission_df.columns = ['image_id','label']

submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)

Thank you.